In [1]:
import numpy as np
import matplotlib.pyplot as plt

G = 6.6743E-11

In [2]:
def findAcceleration(N, r, m):
    a = np.zeros((N, 3))
    for i in range(len(m)):
        for j in range(N):
            if (i != j):
                a[i] = G*m[i]*m[j]*(r[j] - r[i]) / np.linalg.norm(r[j] - r[i])**3
    return a

def f_true(r, v, m, delta_t):
    #need to add in rk4 steps
    return np.array([r, v])

def RK4(u_k, delta_t, t):
    k1 = f_true(u_k, t)
    k2 = f_true(u_k + 0.5*delta_t*k1, t + 0.5*delta_t)
    k3 = f_true(u_k + 0.5*delta_t*k2, t + 0.5*delta_t)
    k4 = f_true(u_k + delta_t*k3, t + delta_t)
    return u_k + (delta_t / 6) * (k1 + 2*k2 + 2*k3 + k4)

m_vec = np.array([3, 1, 2]) #sample mass value array
r_vec = np.array([[0, 0, 0], [1, 1, 1], [2, 2, 3]]) #sample positions value array
v_vec = np.array([[4, 1, 2], [6, 2, 3], [3, 5, 2]]) #sample velocities value array
N_body = 3 #sample N-body number

In [3]:
print(findAcceleration(N_body, r_vec, m_vec))

[[ 1.14265096e-11  1.14265096e-11  1.71397644e-11]
 [ 9.08257188e-12  9.08257188e-12  1.81651438e-11]
 [-9.08257188e-12 -9.08257188e-12 -1.81651438e-11]]
